# Creating IMDB Movie Datbase

Our quest is to create a database for IMDB movies, such that we can save informations from various social media sources using this database schema. We have to make sure our database schema is flexible for any further addition to the list of tables.

This data will further be used with natural language processing for sentiment analysis.

In [76]:
#calling dependencies
import pandas as pd
import numpy as np
import sqlite3

## Datasets

The datasets we are using have been scraped (or API used) to gather the data set from various sources namely:

- [IMDB](http://www.imdb.com/)
- [facebook](https://www.facebook.com/)

The way we scraped the websites has been described in detail in the other notebooks provided.

In [77]:
#reading the movie dataset
movie_data = pd.read_csv("imdb_movie_dataset.csv") 

In [78]:
#reading post dataset
post_data = pd.read_csv("post_data_fb.csv")
post_data = post_data.drop(['Unnamed: 0'], axis = 1)

In [79]:
#reading comment dataset
comment_data = pd.read_csv("comments_posts_fb.csv")
comment_data = comment_data.drop(['Unnamed: 0'], axis = 1)

## Creating master tables abd normalizing the data into 3rd Normal form

We split our data into database tables to fit our database schema. 

In [80]:
#main table - All the other tables will be connected to this one
movie_master_table = pd.DataFrame()
movie_master_table[['movie_id','movie_names','movie_description','imdb_ratings','metascores','runtime','gross_value','year_release']] = movie_data[['Movie_id','movie_names','movie_description','imdb_ratings','metscores','runtime','gross_value','year_release']]
movie_master_table = movie_master_table.drop_duplicates()
movie_master_table.head(3)

,movie_id,movie_names,movie_description,imdb_ratings,metascores,runtime,gross_value,year_release
0,267047309977805,Logan,"[""In the near future, a weary Logan cares for ...",8.1,77,137,226277068.0,2017
16,422679747902228,Wonder Woman,['When a pilot crashes and tells of conflict i...,7.5,76,141,412563408.0,2017
28,387651361687615,Guardians of the Galaxy Vol. 2,"[""The Guardians must fight to keep their newfo...",7.7,67,136,389813101.0,2017


In [81]:
#starcast main table - this will connect to movie table using a seperate relational table
starcast_master_table = pd.DataFrame()
starcast_master_table[['starcast_id','starcast_name']] = movie_data[['star_cast_id','star_cast']]
starcast_master_table = starcast_master_table.drop_duplicates()
starcast_master_table.head(3)

,starcast_id,starcast_name
0,10123,James Mangold
4,10124,Hugh Jackman
8,10125,Patrick Stewart


In [82]:
#director main table - this will connect to movie table using a seperate relational table
director_master_table = pd.DataFrame()
director_master_table[['director_id','director_name']] = movie_data[['director_id','director_name']]
director_master_table = director_master_table.drop_duplicates()
director_master_table.head(3)

,director_id,director_name
0,1200,James Mangold
16,1201,Patty Jenkins
28,1202,James Gunn


In [83]:
#genre main table - this will connect to movie table using a seperate relational table
genre_master_table = pd.DataFrame()
genre_master_table[['genre_id','genre']] = movie_data[['genre_id','genre']]
genre_master_table = genre_master_table.drop_duplicates()
genre_master_table.head(3)

,genre_id,genre
0,100,Action
1,101,Drama
2,102,Sci


In [84]:
#certificate main table - this will connect to movie table using a seperate relational table
certificate_master_table = pd.DataFrame()
certificate_master_table[['certificate_id','certificate']] = movie_data[['certificate_id','certificate']]
certificate_master_table = certificate_master_table.drop_duplicates()
certificate_master_table.head(3)

,certificate_id,certificate
0,1,R
16,2,PG-13
148,3,PG


In [85]:
#posts main table - this will connect to movie table using a seperate relational table (these are posts from facebook)
posts_master_table = pd.DataFrame()
posts_master_table[['post_id','created_at','post_message','likes_count','share_count','comments_count','user_engagement']] = post_data[['post_id','created_at','post_message','post_likes_count','post_shares_count','post_comment_count','user_engagement']]
posts_master_table = posts_master_table.drop_duplicates()
posts_master_table['created_at'] = posts_master_table['created_at'].astype('datetime64[ns]')
posts_master_table.head(3)

,post_id,created_at,post_message,likes_count,share_count,comments_count,user_engagement
0,635054059945646_1531844043599972,2018-02-25 20:00:01,All hail the king. Have you seen Marvel Studio...,4267,223,193,4683
1,635054059945646_1530626643721712,2018-02-24 18:00:01,The people have spoken. Marvel Studios’ “Black...,8952,973,354,10279
2,635054059945646_1529848977132812,2018-02-23 23:54:29,Wakanda Forever. Here's a look at some #BlackP...,3722,503,104,4329


In [86]:
#comments main table - this will connect to posts table using a seperate relational table
comments_master_table = pd.DataFrame()
comments_master_table[['comments_id','created_at','comments']] = comment_data[['comments_id','created_at','post_comments']]
comments_master_table = comments_master_table.drop_duplicates()
comments_master_table['created_at'] = comments_master_table['created_at'].astype('datetime64[ns]')
comments_master_table.head(3)

,comments_id,created_at,comments
0,1531844043599972_1531852783599098,2018-02-25 20:11:22,Just seen it. Probably one of the most disappo...
1,1531844043599972_1531875303596846,2018-02-25 20:36:52,"Take note DC, in just 10 days black panther ha..."
2,1531844043599972_1532535360197507,2018-02-26 14:14:50,"Lol!!! Rotten Tomatoes is funny! To me, Black ..."


## Creating Mappings between all the tables

Here, we create seperate mapping tables between the database tables in order to perform join statements.


In [87]:
#movie-director mapping
movie_director_maping = pd.DataFrame()
movie_director_maping[['movie_id','director_id']] = movie_data[['Movie_id','director_id']]
movie_director_maping = movie_director_maping.drop_duplicates()
movie_director_maping.head(3)

,movie_id,director_id
0,267047309977805,1200
16,422679747902228,1201
28,387651361687615,1202


In [88]:
#movie-starcast mapping
movie_starcast_maping = pd.DataFrame()
movie_starcast_maping[['movie_id','starcast_id']] = movie_data[['Movie_id','star_cast_id']]
movie_starcast_maping = movie_starcast_maping.drop_duplicates()
movie_starcast_maping.head(3)

,movie_id,starcast_id
0,267047309977805,10123
4,267047309977805,10124
8,267047309977805,10125


In [89]:
#movie-genre mapping
movie_genre_maping = pd.DataFrame()
movie_genre_maping[['movie_id','genre_id']] = movie_data[['Movie_id','genre_id']]
movie_genre_maping = movie_genre_maping.drop_duplicates()
movie_genre_maping.head(3)

,movie_id,genre_id
0,267047309977805,100
1,267047309977805,101
2,267047309977805,102


In [90]:
#movie-certificate mapping
movie_certificate_maping = pd.DataFrame()
movie_certificate_maping[['movie_id','certificate_id']] = movie_data[['Movie_id','certificate_id']]
movie_certificate_maping = movie_certificate_maping.drop_duplicates()
movie_certificate_maping.head(3)

,movie_id,certificate_id
0,267047309977805,1
16,422679747902228,2
28,387651361687615,2


In [91]:
#movie-post mapping
movie_post_maping = pd.DataFrame()
movie_post_maping[['movie_id','post_id']] = post_data[['movie_id','post_id']]
movie_post_maping = movie_post_maping.drop_duplicates()
movie_post_maping.head(3)

,movie_id,post_id
0,6.350540e+14,635054059945646_1531844043599972
1,6.350540e+14,635054059945646_1530626643721712
2,6.350540e+14,635054059945646_1529848977132812


In [92]:
#post-comment mapping
post_comment_maping = pd.DataFrame()
post_comment_maping[['post_id','comments_id']] = comment_data[['post_id','comments_id']]
post_comment_maping = post_comment_maping.drop_duplicates()
post_comment_maping.head(3)

,post_id,comments_id
0,635054059945646_1531844043599972,1531844043599972_1531852783599098
1,635054059945646_1531844043599972,1531844043599972_1531875303596846
2,635054059945646_1531844043599972,1531844043599972_1532535360197507


## Creating databases

Now that we have our schma and the tables for the database ready, let us start creating the tables in the databases and start storing the vales in them.

In [93]:
#creating connections
conn = sqlite3.connect("imdb_movie.db") #creates and connects to a database named "imdb_movie"
c = conn.cursor()

In [94]:
c.execute("""DROP TABLE movie_master_table""")
c.execute("""CREATE TABLE movie_master_table(
movie_id INTEGER PRIMARY KEY,
movie_names CHAR(50),
movie_description VARCHAR(200),
imdb_ratings FLOAT,
metascores INTEGER,
runtime INTEGER,
gross_value INTEGER,
year_release CHAR(4));""") #creating a new tables within database

In [95]:
movie_master_table.to_sql("movie_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [96]:
c.execute("""DROP TABLE starcast_master_table;""")
c.execute("""CREATE TABLE starcast_master_table(
starcast_id INTEGER PRIMARY KEY,
starcast_name CHAR NOT NULL);""") #creating a new tables within database

In [97]:
starcast_master_table.to_sql("starcast_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [98]:
c.execute("""DROP TABLE director_master_table;""")
c.execute("""CREATE TABLE director_master_table(
director_id INTEGER PRIMARY KEY,
director_name CHAR NOT NULL);""") #creating a new tables within database

In [99]:
director_master_table.to_sql("director_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [100]:
c.execute("""DROP TABLE genre_master_table;""")
c.execute("""CREATE TABLE genre_master_table(
genre_id INTEGER PRIMARY KEY,
genre CHAR NOT NULL);""") #creating a new tables within database

In [101]:
genre_master_table.to_sql("genre_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [102]:
c.execute("""DROP TABLE certificate_master_table;""")
c.execute("""CREATE TABLE certificate_master_table(
certificate_id INTEGER PRIMARY KEY,
certificate CHAR NOT NULL);""") #creating a new tables within database

In [103]:
certificate_master_table.to_sql("certificate_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [110]:
c.execute("""DROP TABLE posts_master_table;""")
c.execute("""CREATE TABLE posts_master_table(
post_id CHAR PRIMARY KEY,
created_at DATE,
post_message CHAR,
likes_count INTEGER,
share_count INTEGER,
comments_count INTEGER,
user_engagement INTEGER) ;""") #creating a new tables within database

In [111]:
posts_master_table.to_sql("posts_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [112]:
c.execute("""DROP TABLE comments_master_table;""")
c.execute("""CREATE TABLE comments_master_table(
comments_id CHAR PRIMARY KEY,
created_at DATE,
comments CHAR);""") #creating a new tables within database

In [113]:
comments_master_table.to_sql("comments_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

## Creating connection to the database

Here, we are using SQLite database management system in order to create store and analyse our data. we create a relational database schema and store our data inside.


In [114]:
c.execute("""DROP TABLE movie_director_maping;""")
c.execute("""CREATE TABLE movie_director_maping(
movie_id INTEGER FOREGION KEY,
director_id INTEGER FOREGION KEY)""") #creating a new tables within database

In [115]:
movie_director_maping.to_sql("movie_director_maping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [116]:
c.execute("""DROP TABLE movie_starcast_maping;""")
c.execute("""CREATE TABLE movie_starcast_maping(
movie_id INTEGER FOREGION KEY,
starcast_id INTEGER FOREGION KEY)""") #creating a new tables within database

In [117]:
movie_starcast_maping.to_sql("movie_starcast_maping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [118]:
c.execute("""DROP TABLE movie_genre_maping;""")
c.execute("""CREATE TABLE movie_genre_maping(
movie_id INTEGER FOREGION KEY,
genre_id INTEGER FOREGION KEY)""") #creating a new tables within database

In [119]:
movie_genre_maping.to_sql("movie_genre_maping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [120]:
c.execute("""DROP TABLE movie_certificate_maping;""")
c.execute("""CREATE TABLE movie_certificate_maping(
movie_id INTEGER FOREGION KEY,
certificate_id INTEGER FOREGION KEY)""") #creating a new tables within database

In [121]:
movie_certificate_maping.to_sql("movie_certificate_maping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [122]:
c.execute("""DROP TABLE movie_post_maping;""")
c.execute("""CREATE TABLE movie_post_maping(
movie_id INTEGER FOREGION KEY,
post_id CHAR FOREGION KEY)""") #creating a new tables within database

In [123]:
movie_post_maping.to_sql("movie_post_maping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [124]:
c.execute("""DROP TABLE post_comment_maping;""")
c.execute("""CREATE TABLE post_comment_maping(
post_id CHAR FOREGION KEY,
comments_id CHAR FOREGION KEY)""") #creating a new tables within database

In [125]:
post_comment_maping.to_sql("post_comment_maping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

## Answerig Questions

Now that we have aggregated our dataset, let us go ahead and try to answer certain basic questions on the basis of data we have collected. The way we will be answering these questions may not be best way to go about in real life, but it gives us a good start to getting insights.

The quetions we will be addressing here are:
- What tags are associated with person, place or thing?
- What social media users are like other social media useres in our domain?
- What people, places or things are famous in our domain?
- What people places or things are trending in our domain?

 We will answer all this questions by implementing our usecases using SQL queries 

# Question 1:
## What tags are associated with person, place or thing?

I have chosen the movie names as tags for my database

In [141]:
pd.read_sql_query("""SELECT starcast_name, movie_names
FROM movie_master_table mmt
JOIN movie_starcast_maping msm ON mmt.movie_id = msm.movie_id
JOIN starcast_master_table smt ON msm.starcast_id = smt.starcast_id;""",conn).head()

,starcast_name,movie_names
0,James Mangold,Logan
1,Hugh Jackman,Logan
2,Patrick Stewart,Logan
3,Dafne Keen,Logan
4,Patty Jenkins,Wonder Woman


# Question 3:
## What people places or things are famous in our domain?

To answer this question we check the posts of what movies(my tags) are being liked the most.

In [127]:
pd.read_sql_query(""" SELECT movie_names, likes_count
FROM movie_master_table mmt
JOIN movie_post_maping mpm ON mmt.movie_id = mpm.movie_id
JOIN posts_master_table pmt ON mpm.post_id = pmt.post_id
ORDER BY pmt.likes_count DESC;""", conn).head()

,movie_names,likes_count
0,To the Bone,24477
1,Beauty and the Beast,19975
2,To the Bone,17715
3,To the Bone,14556
4,To the Bone,14423


# Question 2:
## What people, place or thing is trending in our domain?

To answer the question we check for user engagement with a particular post and the movie associated with that post. Engagement is the sum of counts of likes comments and shares a post gets.

In [132]:
pd.read_sql_query(""" SELECT movie_names, user_engagement, created_at
FROM movie_master_table mmt
JOIN movie_post_maping mpm ON mmt.movie_id = mpm.movie_id
JOIN posts_master_table pmt ON mpm.post_id = pmt.post_id
ORDER BY pmt.user_engagement DESC;""", conn).head()

,movie_names,user_engagement,created_at
0,Beauty and the Beast,28968,2018-02-12 16:30:01
1,To the Bone,27843,2017-04-07 18:01:04
2,To the Bone,21302,2017-04-06 17:00:00
3,To the Bone,15970,2017-04-03 18:45:00
4,To the Bone,15440,2017-06-13 16:00:00


# Question 2:
## what people are like other people in our domain?

For this we see if the directors are working in the same genre. IF they work in the same domain we tag them as being similar

In [142]:
pd.read_sql_query("""SELECT genre, starcast_name
FROM starcast_master_table smt, genre_master_table gmt
JOIN movie_starcast_maping msm ON msm.starcast_id = smt.starcast_id
JOIN movie_genre_maping mgm ON mgm.genre_id = gmt.genre_id""",conn).head()

,genre,starcast_name
0,Action,James Mangold
1,Drama,James Mangold
2,Sci,James Mangold
3,Fi,James Mangold
4,Action,James Mangold
